In [1]:
from simpletransformers.classification import MultiLabelClassificationArgs, MultiLabelClassificationModel
import sklearn
import pandas as pd
import logging
from util import *
from sklearn.preprocessing import MultiLabelBinarizer
#from transformers imp ort *
import numpy as np
import os
import re
import tensorflow as tf
import torch
from util import *
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [2]:
training_set = pd.read_json("training_set.json.gz", lines=True, orient="records")
testing_set = pd.read_json("testing_set.json.gz", lines=True, orient="records")
subset = list(set(all_tiers_100)-set(["PersonalizedProduct"]))

In [3]:
subset

['AnatomicalTarget_LowerExtremity',
 'Imaging',
 'SpecificationofUse_JointReplacement',
 'AnatomicalTarget_Torso',
 'AnatomicalTarget',
 'AnatomicalTarget_UpperExtremity',
 'Imaging_CT',
 'SurgicalMethod',
 'AnatomicalTarget_UpperExtremity_Shoulder',
 'AnatomicalTarget_LowerExtremity_Hip',
 'Imaging_MRI',
 'AnalysisAndModeling_3DModeling',
 'Imaging_Ultrasound',
 'AnatomicalTarget_LowerExtremity_Knee',
 'Manufacturing_AdditiveManufacturing',
 'AnatomicalTarget_Torso_Spine',
 'PersonalizedProduct_Guide/Jig',
 'Manufacturing',
 'AnalysisAndModeling',
 'SpecificationofUse_Disease',
 'PersonalizedProduct_Implant',
 'SpecificationofUse']

In [4]:
training_set['labels'] = training_set[subset].apply(array_labels,axis=1)
testing_set['labels'] = testing_set[subset].apply(array_labels,axis=1)

In [5]:
mlb = MultiLabelBinarizer()
training_labels  = mlb.fit_transform(training_set.labels)
testing_labels = mlb.transform(testing_set.labels)

In [6]:
training_set[subset]

,AnatomicalTarget_LowerExtremity,Imaging,SpecificationofUse_JointReplacement,AnatomicalTarget_Torso,AnatomicalTarget,AnatomicalTarget_UpperExtremity,Imaging_CT,SurgicalMethod,AnatomicalTarget_UpperExtremity_Shoulder,AnatomicalTarget_LowerExtremity_Hip,...,Imaging_Ultrasound,AnatomicalTarget_LowerExtremity_Knee,Manufacturing_AdditiveManufacturing,AnatomicalTarget_Torso_Spine,PersonalizedProduct_Guide/Jig,Manufacturing,AnalysisAndModeling,SpecificationofUse_Disease,PersonalizedProduct_Implant,SpecificationofUse
0,True,True,True,True,True,True,True,False,True,True,...,False,True,False,True,True,True,True,False,False,True
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
2,True,False,True,False,True,False,False,False,False,False,...,False,True,False,False,False,True,True,False,True,True
3,False,True,False,False,True,True,False,False,True,False,...,False,False,False,False,True,True,True,False,False,False
4,True,False,False,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,True,True,False,False,True,True,True,False,True,True,...,False,True,False,False,True,False,True,False,True,False
968,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,False,True,False,False,False,False
969,False,True,False,False,False,False,False,False,False,False,...,False,False,True,False,True,True,False,False,False,False
970,True,True,False,False,True,False,True,False,False,True,...,False,False,False,False,True,False,False,False,False,False


In [7]:
train_df = pd.DataFrame()
train_df["text"] = training_set.abstract
train_df["labels"] = [list(x) for x in training_labels]

In [8]:
test_df = pd.DataFrame()
test_df["text"] = testing_set.description
test_df['labels'] = [list(x) for x in testing_labels]

In [9]:
train_df.labels

0      [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, ...
1      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2      [1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
3      [1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, ...
4      [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
                             ...                        
967    [1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, ...
968    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
969    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
970    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, ...
971    [0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, ...
Name: labels, Length: 972, dtype: object

In [10]:
test_df.labels

0      [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
1      [1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
2      [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3      [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, ...
4      [1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, ...
                             ...                        
238    [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, ...
239    [1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...
240    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...
241    [0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
242    [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...
Name: labels, Length: 243, dtype: object

In [11]:
def hammer(true, pred):
    hamming_loss(true, pred > 0.5)

In [12]:
model_args = MultiLabelClassificationArgs()
model_args.fp16 = True

model_args.do_lower_case = False
model_args.max_seq_length=128

model_args.warmup_steps=1000
model_args.learning_rate = 5e-05
model_args.gradient_accumulation_steps = 4
#gradient_checkpointing=True
#model_args.weight_decay = 0.01

model_args.train_batch_size= 8
model_args.num_train_epochs = 50

model_args.overwrite_output_dir = True
model_args.save_best_model=True
model_args.save_model_every_epoch=False
model_args.save_eval_checkpoints=False

model_args.use_early_stopping = True
model_args.early_stopping_delta = 0.001
model_args.early_stopping_metric = "eval_loss"
model_args.early_stopping_metric_minimize = True
model_args.early_stopping_patience = 5

model_args.evaluate_during_training_steps = 50
model_args.evaluate_during_training=True
model_args.evaluate_during_training_silent=False
model_args.evaluate_during_training_verbose=True
model_args.use_cached_eval_features=True
model_args.evaluate_each_epoch=False
model_args.dataloader_num_workers=0


In [13]:
#model = BertForPreTraining.from_pretrained("/var/patentmark/patentBERT")

In [14]:
#model.save_pretrained("/var/patentmark/conv-patentBERT2/")

In [15]:
model = MultiLabelClassificationModel(
    "bert", "bertForPatents/", args=model_args, num_labels=len(mlb.classes_)
)

Some weights of the model checkpoint at bertForPatents/ were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSe

In [16]:
model.train_model(train_df, eval_df=test_df, show_running_loss=True)

INFO:simpletransformers.classification.classification_model: Converting to features started. Cache is not used.


INFO:simpletransformers.classification.classification_model:   Starting fine-tuning.


INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_bert_128_22_243


INFO:simpletransformers.classification.classification_model:{'LRAP': 0.5652158772745105, 'eval_loss': 0.6094606153426632}


/home/martin/anaconda3/envs/phenetics2/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)
INFO:simpletransformers.classification.classification_model: No improvement in eval_loss
INFO:simpletransformers.classification.classification_model: Current step: 1
INFO:simpletransformers.classification.classification_model: Early stopping patience: 5


...

INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_bert_128_22_243


INFO:simpletransformers.classification.classification_model:{'LRAP': 0.5915878790015311, 'eval_loss': 0.5439359641844227}
INFO:simpletransformers.classification.classification_model: No improvement in eval_loss
INFO:simpletransformers.classification.classification_model: Current step: 2
INFO:simpletransformers.classification.classification_model: Early stopping patience: 5


INFO:simpletransformers.classification.classification_model: Features loaded from cache at cache_dir/cached_dev_bert_128_22_243


INFO:simpletransformers.classification.classification_model:{'LRAP': 0.5777766493321287, 'eval_loss': 0.5462702868446228}
INFO:simpletransformers.classification.classification_model: No improvement in eval_loss
INFO:simpletransformers.classification.classification_model: Current step: 3
INFO:simpletransformers.classification.classification_model: Early stopping patience: 5


KeyboardInterrupt: 

In [22]:
from sklearn.metrics import *

In [17]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

In [18]:
result

{'LRAP': 0.5627058445520303, 'eval_loss': 0.5878909105254758}

In [19]:
import numpy as np
np.mean(model_outputs)

0.23355764366917695

In [20]:
sklearn.metrics.hamming_loss(testing_labels, model_outputs > 0.5)

0.27983539094650206

In [24]:
#Abstract - 
print(classification_report(testing_labels, model_outputs > .5, target_names=mlb.classes_))

                                               precision    recall  f1-score   support

                        Analysis and Modeling       0.40      0.27      0.33        84
           Analysis and Modeling: 3D Modeling       0.41      0.15      0.22        71
                            Anatomical Target       0.67      0.79      0.73       164
           Anatomical Target: Lower Extremity       0.52      0.28      0.37       113
     Anatomical Target: Lower Extremity - Hip       0.00      0.00      0.00        40
    Anatomical Target: Lower Extremity - Knee       0.50      0.01      0.02        82
                     Anatomical Target: Torso       0.00      0.00      0.00        35
             Anatomical Target: Torso - Spine       0.00      0.00      0.00        21
           Anatomical Target: Upper Extremity       0.00      0.00      0.00        31
Anatomical Target: Upper Extremity - Shoulder       0.00      0.00      0.00        23
                                      Imag

In [ ]:
# import numpy as np
# import os
# import re
# import tensorflow as tf
# import torch
# def convert_tf_checkpoint_to_pytorch(tf_checkpoint_path, bert_config_file, pytorch_dump_path):
#     config_path = os.path.abspath(bert_config_file)
#     tf_path = os.path.abspath(tf_checkpoint_path)
#     print("Converting TensorFlow checkpoint from {} with config at {}".format(tf_path, config_path))
#     # Load weights from TF model
#     init_vars = tf.train.list_variables(tf_path)
#     excluded = ["BERTAdam", "_power", "global_step", "_CHECKPOINTABLE_OBJECT_GRAPH"]
#     init_vars = list(filter(lambda x: all([True if e not in x[0] else False for e in excluded]), init_vars))
#     names = []
#     arrays = []
#     for name, shape in init_vars:
#         print("Loading TF weight {} with shape {}".format(name, shape))
#         array = tf.train.load_variable(tf_path, name)
#         names.append(name)
#         arrays.append(array)

#     config = BertConfig.from_json_file(bert_config_file)
#     print("Building PyTorch model from configuration: {}".format(str(config)))
#     # Initialise PyTorch model
#     model = BertForSequenceClassification(config)

#     for name, array in zip(names, arrays):
#         name = name.split("/")
#         # adam_v and adam_m are variables used in AdamWeightDecayOptimizer to calculated m and v
#         # which are not required for using pretrained model
#         if any(n in ["adam_v", "adam_m", "global_step", "bad_steps", "global_step", "good_steps", "loss_scale",
#                      "AdamWeightDecayOptimizer", "AdamWeightDecayOptimizer_1", "save_counter", ".OPTIMIZER_SLOT"] for n in name) or \
#                 name[0] == "optimizer":
#             print("Skipping {}".format("/".join(name)))
#             continue
#         if ".OPTIMIZER_SLOT" in name:
#             idx = name.index(".OPTIMIZER_SLOT")
#             name = name[:idx]
#         elif ".ATTRIBUTES" in name:
#             idx = name.index(".ATTRIBUTES")
#             name = name[:idx]
#         print(name)
#         pointer = model
#         for m_name in name:
#             try:
#                 if re.fullmatch(r"[A-Za-z]+_\d+", m_name):
#                     scope_names = re.split(r"_(\d+)", m_name)
#                 else:
#                     scope_names = [m_name]
#                 if scope_names[0] == "kernel" or scope_names[0] == "gamma":
#                     pointer = getattr(pointer, "weight")
#                 elif scope_names[0] == "output_bias" or scope_names[0] == "beta":
#                     pointer = getattr(pointer, "bias")
#                 elif scope_names[0] == "output_weights":
#                     pointer = getattr(pointer, "weight")
#                 elif scope_names[0] == "squad":
#                     pointer = getattr(pointer, "classifier")
#                 elif scope_names[0] == "dense_output" or scope_names[0] == "bert_output":
#                     pointer = getattr(pointer, "output")
#                 elif scope_names[0] == "self_attention":
#                     pointer = getattr(pointer, "self")
#                 else:
#                     pointer = getattr(pointer, scope_names[0])
#             except AttributeError:
#                     logger.info("Skipping {}".format("/".join(name)))
#                     continue
#             if len(scope_names) >= 2:
#                 num = int(scope_names[1])
#                 pointer = pointer[num]
#         if m_name[-11:] == "_embeddings":
#             pointer = getattr(pointer, "weight")
#         elif m_name == "kernel" or m_name == "gamma" or m_name == "output_weights":
#             array = np.transpose(array)
#         # print("Initialize PyTorch weight {}".format(name))
#         pointer.data = torch.from_numpy(array)

#     # Save pytorch-model
#     print("Save PyTorch model to {}".format(pytorch_dump_path))
#     torch.save(model.state_dict(), pytorch_dump_path)


# convert_tf_checkpoint_to_pytorch("/var/patentmark/patentBERT/model.ckpt", "/var/patentmark/patentBERT/config.json", pytorch_dump_path="/var/patentmark/patentBERT/pytorch.bin")